In [19]:
from dotenv import load_dotenv
load_dotenv()
from typing import Annotated

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [11]:
from langchain.chat_models import init_chat_model

In [13]:
pip install -U langchain-groq

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   -------------------- ------------------- 1/2 [langchain-groq]
   ---------------------------------------- 2/2 [langchain-groq]

Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_core import messages


llm = init_chat_model("groq:llama-3.1-8b-instant") # create a llm model

# setup initial state
class State(TypedDict):
    messages: Annotated[list, add_messages] # automatically give the message with answer

# create function for the first node
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

# create the graph
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [21]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='51329300-1f87-44e8-996b-73d023cc25f4'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 49, 'total_tokens': 52, 'completion_time': 0.004102559, 'completion_tokens_details': None, 'prompt_time': 0.002368002, 'prompt_tokens_details': None, 'queue_time': 0.056343998, 'total_time': 0.006470561}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b81c4-9537-7c70-a560-ebee27ac5114-0', usage_metadata={'input_tokens': 49, 'output_tokens': 3, 'total_tokens': 52})]